In [1]:
import sys
import random
import time
import numpy as np
import pandas as pd

# **Task 1**

Задамо тестову пару значень p1 та p2, аби порівнювати між собою результати трьох алгоритмів.

In [2]:
p1=0.65
p2=0.3

## Monte-Carlo method

Для даного алгоритму треба прописати симуляцію однієї партії в теніс.\
Просимулювавши величезну кількість партій ми очікуємо, що емпіричні ймовірності наближено дорівнюють теоретичним.

На кожній подачі важливо знати, хто подає (цикл подач - 4, тобто по дві від кожного гравця).

In [3]:
def end_of_game(score):
    return score[0]==11 or score[1]==11 or (score[0]==10 and score[1]==10)

def run_simulation(p1, p2):
    i = 0
    score = [0, 0]
    while not end_of_game(score):
        if i%4<2:
            if p1>random.uniform(0, 1):
                score[0] += 1
            else:
                score[1] += 1
        else:
            if p2>random.uniform(0, 1):
                score[0] += 1
            else:
                score[1] += 1
        i += 1
    return tuple(score)

def monte_carlo(p1, p2, n_simulations=100000):
    start = time.time()
    dict_scores = dict()
    for score in [(i, 11) for i in range(10)]+[(10, 10)]+[(11, i) for i in range(10)]:
        dict_scores[score] = 0
    for i in range(n_simulations):
        sc = run_simulation(p1, p2)
        dict_scores[sc] += 1
    return sys.getsizeof(dict_scores), time.time()-start, dict(zip(dict_scores.keys(), np.array(list(dict_scores.values()))/n_simulations))

In [4]:
memory, seconds, probs = monte_carlo(p1, p2)
print("Monte-Carlo method")
print(f"Memory usage: {memory} bytes")
print(f"Time usage:   {seconds:.2f} sec")
probs

Monte-Carlo method
Memory usage: 656 bytes
Time usage:   1.55 sec


{(0, 11): 0.00027,
 (1, 11): 0.00293,
 (2, 11): 0.00625,
 (3, 11): 0.01187,
 (4, 11): 0.03803,
 (5, 11): 0.07057,
 (6, 11): 0.05126,
 (7, 11): 0.05803,
 (8, 11): 0.12707,
 (9, 11): 0.13474,
 (10, 10): 0.1832,
 (11, 0): 0.00015,
 (11, 1): 0.00077,
 (11, 2): 0.00493,
 (11, 3): 0.01559,
 (11, 4): 0.01621,
 (11, 5): 0.02233,
 (11, 6): 0.0602,
 (11, 7): 0.0892,
 (11, 8): 0.05182,
 (11, 9): 0.05458}

## Tree method

При побудові дерева всіх можливих шляхів розвитку партії, ми для кожного проміжного кроку зберігаємо проміжний рахунок та ймовірність дійти до цього рахунку конкретною послідовністю виграшів.

Вузол вважаємо листком, якщо рахунок стає кінцевим в партії, і додаємо відповідну ймовірність у фінальний словник.

Отже, ми проходимо через усе дерево тільки один раз (під час його побудови).

In [5]:
class Node:
    def __init__(self, score, prob, i):
        self.score = score
        self.prob = prob
        self.i = i
        self.left = None
        self.right = None
    
    def end_of_game(self):
        return self.score[0]==11 or self.score[1]==11 or (self.score[0]==10 and self.score[1]==10)
    
    def next_step(self, p1, p2):
        if self.i%4<2:
            self.left = Node((self.score[0]+1, self.score[1]), self.prob*p1, self.i+1)
            self.right= Node((self.score[0], self.score[1]+1), self.prob*(1-p1), self.i+1)
        else:
            self.left = Node((self.score[0]+1, self.score[1]), self.prob*p2, self.i+1)
            self.right= Node((self.score[0], self.score[1]+1), self.prob*(1-p2), self.i+1)


class Tree:
    def __init__(self, p1, p2):
        self.root = Node((0, 0), 1, 0)
        self.p1 = p1
        self.p2 = p2
        self.dict_scores = dict()
        self.memory_usage = 0

    def build(self, root):
        if root:
            if root.end_of_game():
                try:
                    self.dict_scores[root.score] += root.prob
                except:
                    self.dict_scores[root.score] = root.prob
            else:
                root.next_step(self.p1, self.p2)
            self.memory_usage += sys.getsizeof(root)
            self.build(root.left)
            self.build(root.right)

    def get_scores(self):
        return self.dict_scores

def tree(p1, p2):
    start = time.time()
    tree = Tree(p1, p2)
    tree.build(tree.root)
    return tree.memory_usage, time.time()-start, tree.get_scores()

In [6]:
memory, seconds, probs = tree(p1, p2)
print("Tree method")
print(f"Memory usage: {memory} bytes")
print(f"Time usage:   {seconds:.2f} sec")
probs

Tree method
Memory usage: 66646464 bytes
Time usage:   9.11 sec


{(11, 0): 0.00018326790421875,
 (11, 1): 0.0008190665565468748,
 (11, 2): 0.004690389570201557,
 (11, 3): 0.01515941430905176,
 (11, 4): 0.01623989735202245,
 (11, 5): 0.02222224816761459,
 (11, 6): 0.06144065673641841,
 (11, 7): 0.08920710980220121,
 (11, 8): 0.051454278847830354,
 (11, 9): 0.05387497301495137,
 (10, 10): 0.18356375847164116,
 (9, 11): 0.13499613890964157,
 (8, 11): 0.12591605071778864,
 (7, 11): 0.05860898046776838,
 (6, 11): 0.052224402860902776,
 (5, 11): 0.0701347051751836,
 (4, 11): 0.03792482569736893,
 (3, 11): 0.011864114936651734,
 (2, 11): 0.006293460274204677,
 (1, 11): 0.0028733029234218727,
 (0, 11): 0.0003089573035937498}

## Markov process

Стани - всі можливі проміжні рахунки в партії. Всього їх 141 стан.\
Термінальні стани - кінцеві рахунки.

Початковий стан - рахунок (0, 0).

Матрицю переходів підносимо до 20 степені, бо партія гарантовано закінчується за <= 20 подач.

In [7]:
def score_to_pos(score):
    if score[0]<=10:
        return score[0]*12 + score[1]
    else:
        return 131 + score[1]

def get_transition_matrix(p1, p2):
    transition_matrix = np.zeros((141, 141))

    for i in range(11):
        for j in range(11):
            pos = score_to_pos([i, j])
            if i+j==20 or i==11 or j==11:
                transition_matrix[pos, pos] = 1
            elif (i+j)%4<2:
                transition_matrix[pos, score_to_pos([i+1, j])] = p1
                transition_matrix[pos, score_to_pos([i, j+1])] = 1-p1
            else:
                transition_matrix[pos, score_to_pos([i+1, j])] = p2
                transition_matrix[pos, score_to_pos([i, j+1])] = 1-p2

    for i in range(10):
        pos = score_to_pos([i, 11])
        transition_matrix[pos, pos] = 1
        pos = score_to_pos([11, i])
        transition_matrix[pos, pos] = 1

    return transition_matrix

def markov_process(p1, p2):
    start = time.time()
    transition_matrix = get_transition_matrix(p1, p2)
    init_states = np.zeros((1, 141))
    init_states[0, 0] = 1
    prob_matrix = np.dot(init_states, np.linalg.matrix_power(transition_matrix, 20))
    return (sys.getsizeof(transition_matrix)+sys.getsizeof(init_states)+sys.getsizeof(prob_matrix),
            time.time()-start,
            dict(zip([(i, 11) for i in range(10)]+[(10, 10)]+[(11, i) for i in range(10)],
                     [prob_matrix[0, score_to_pos([i, 11])] for i in range(10)]+[prob_matrix[0, score_to_pos([10, 10])]]+[prob_matrix[0, score_to_pos([11, i])] for i in range(10)])))

In [8]:
memory, seconds, probs = markov_process(p1, p2)
print("Markov process method")
print(f"Memory usage: {memory} bytes")
print(f"Time usage:   {seconds:.4f} sec")
probs

Markov process method
Memory usage: 161664 bytes
Time usage:   0.0079 sec


{(0, 11): 0.0003089573035937497,
 (1, 11): 0.0028733029234218727,
 (2, 11): 0.006293460274204682,
 (3, 11): 0.011864114936651788,
 (4, 11): 0.037924825697368963,
 (5, 11): 0.07013470517518669,
 (6, 11): 0.05222440286091093,
 (7, 11): 0.058608980467790575,
 (8, 11): 0.1259160507177725,
 (9, 11): 0.13499613890975012,
 (10, 10): 0.1835637584723195,
 (11, 0): 0.00018326790421875004,
 (11, 1): 0.0008190665565468752,
 (11, 2): 0.004690389570201564,
 (11, 3): 0.015159414309051801,
 (11, 4): 0.01623989735202245,
 (11, 5): 0.02222224816761655,
 (11, 6): 0.061440656736406994,
 (11, 7): 0.08920710980219736,
 (11, 8): 0.051454278847848214,
 (11, 9): 0.05387497301491754}

## Comparison

Для порівняння використовуємо такі показники:
- точність отриманих ймовірностей
- час роботи алгоритму
- кількість використаної пам'яті

In [9]:
def relative_error(true, est):
    return np.array([abs(true[key]-est[key])/true[key] for key in true.keys()]).mean()

In [10]:
print(f"Tree vs. Markov process: {relative_error(tree(p1, p2)[2], markov_process(p1, p2)[2])}")

Tree vs. Markov process: 3.1002320845635384e-13


Методи побудови дерева та марківського процесу дають теоретичні ймовірності. Тому вони завжди точні.\
Метод Монте-Карло дає наближені ймовірності. Тому там завжди буде похибка.

In [11]:
for n in [100000, 1000000, 10000000]:
    print(f"Tree vs. Monte-Carlo(n_simulations={n}): {relative_error(tree(p1, p2)[2], monte_carlo(p1, p2, n)[2]):.4f}")

Tree vs. Monte-Carlo(n_simulations=100000): 0.0473
Tree vs. Monte-Carlo(n_simulations=1000000): 0.0077
Tree vs. Monte-Carlo(n_simulations=10000000): 0.0037


Як бачимо, для методу Монте-Карло важлива кількість симуляцій.\
Збільшення кількості симуляцій на порядок покращує точність.

In [12]:
mc_results = []
tr_results = []
mp_results = []
for p1 in np.arange(0.1, 1, 0.1):
    for p2 in np.arange(0.1, 1, 0.1):
        mc = monte_carlo(p1, p2)
        tr = tree(p1, p2)
        mp = markov_process(p1, p2)
        mc_results.append((relative_error(tr[2], mc[2]), mc[0], mc[1]))
        tr_results.append((relative_error(tr[2], mp[2]), tr[0], tr[1]))
        mp_results.append((relative_error(tr[2], mp[2]), mp[0], mp[1]))

In [13]:
mc_results

[(0.5659837530926597, 656, 1.0855991840362549),
 (0.4571819141226848, 656, 1.1845979690551758),
 (0.33178701476776046, 656, 1.2321419715881348),
 (0.2708512059000543, 656, 1.2923378944396973),
 (0.25293674987012865, 656, 1.3659846782684326),
 (0.22060365336621995, 656, 1.4418628215789795),
 (0.14670805520514554, 656, 1.5411021709442139),
 (0.10863582983423142, 656, 1.6258089542388916),
 (0.2514917877461847, 656, 1.6447162628173828),
 (0.4122135757824673, 656, 1.2017507553100586),
 (0.5452399651837807, 656, 1.2467365264892578),
 (0.2727117342708115, 656, 1.317286729812622),
 (0.1216361293295323, 656, 1.3963825702667236),
 (0.08303163907577604, 656, 1.4743928909301758),
 (0.0785544712726892, 656, 1.5189480781555176),
 (0.04128963449186162, 656, 1.5939643383026123),
 (0.06400428322540741, 656, 1.6447856426239014),
 (0.13743577711853613, 656, 1.665649652481079),
 (0.3252260039739112, 656, 1.2431929111480713),
 (0.36363584279297795, 656, 1.3348228931427002),
 (0.09604187447813928, 656, 1.39

In [14]:
pd.options.display.float_format = '{:.3f}'.format
comparison_table = pd.DataFrame(columns=['Relative error', 'Memory (bytes)', 'Time (sec)'])
comparison_table.loc['Monte-Carlo'] = np.array([np.mean(np.array(i)) for i in zip(mc_results[0])])
comparison_table.loc['Tree'] = np.array([np.mean(np.array(i)) for i in zip(tr_results[0])])
comparison_table.loc['Markov process'] = np.array([np.mean(np.array(i)) for i in zip(mp_results[0])])
comparison_table

,Relative error,Memory (bytes),Time (sec)
Monte-Carlo,0.566,656.000,1.086
Tree,0.000,66646464.000,5.976
Markov process,0.000,161664.000,0.002


# **Task 2**

У попередньому завданні партія гарантовано закінчувалася за <= 20 подач.\
У цьому завданні партія може тривати нескінченно.\
Тому я визначаю 20 подач для досягнення кінця гри. Інакше фіксуємо нічию.\
Також будемо зберігати не фінальний рахунок, а тільки переможця (1 чи 2). 0 - нічия.

In [15]:
p=0.4

## Monte-Carlo method

In [16]:
def end_of_game(score):
    return abs(score[0]-score[1])>=2

def run_simulation(p):
    i = 20
    score = [10, 10]
    while not end_of_game(score) and i<=40:
        if p>random.uniform(0, 1):
            score[0] += 1
        else:
            score[1] += 1
        i += 1
    if (score[0]-score[1])>=2:
        return 1
    elif (score[1]-score[0])>=2:
        return 2
    else:
        return 0

def monte_carlo(p, n_simulations=1000000):
    start = time.time()
    dict_scores = dict()
    for score in [0, 1, 2]:
        dict_scores[score] = 0
    for i in range(n_simulations):
        sc = run_simulation(p)
        dict_scores[sc] += 1
    return sys.getsizeof(dict_scores), time.time()-start, dict(zip(dict_scores.keys(), np.array(list(dict_scores.values()))/n_simulations))

In [17]:
memory, seconds, probs = monte_carlo(p)
print("Monte-Carlo method")
print(f"Memory usage: {memory} bytes")
print(f"Time usage:   {seconds:.2f} sec")
probs

Monte-Carlo method
Memory usage: 248 bytes
Time usage:   3.18 sec


{0: 0.000645, 1: 0.307705, 2: 0.69165}

## Tree method

In [18]:
class Node:
    def __init__(self, score, prob, i):
        self.score = score
        self.prob = prob
        self.i = i
        self.left = None
        self.right = None
    
    def end_of_game(self):
        return abs(self.score[0]-self.score[1])>=2
    
    def next_step(self, p):
        self.left = Node((self.score[0]+1, self.score[1]), self.prob*p, self.i+1)
        self.right= Node((self.score[0], self.score[1]+1), self.prob*(1-p), self.i+1)

    def get_winner(self):
        if (self.score[0]-self.score[1])>=2:
            return 1
        elif (self.score[1]-self.score[0])>=2:
            return 2
        else:
            return 0

class Tree:
    def __init__(self, p):
        self.root = Node((10, 10), 1, 20)
        self.p = p
        self.dict_scores = dict()
        self.memory_usage = 0

    def build(self, root):
        if root:
            if root.end_of_game() or root.i>=40:
                try:
                    self.dict_scores[root.get_winner()] += root.prob
                except:
                    self.dict_scores[root.get_winner()] = root.prob
            else:
                root.next_step(self.p)
            self.memory_usage += sys.getsizeof(root)
            self.build(root.left)
            self.build(root.right)

    def get_scores(self):
        return self.dict_scores
    
    def get_memory_usage(self):
        return self.memory_usage

def tree(p, memory=False):
    start = time.time()
    tree = Tree(p)
    tree.build(tree.root)
    return tree.memory_usage, time.time()-start, tree.get_scores()

In [19]:
memory, seconds, probs = tree(p)
print("Tree method")
print(f"Memory usage: {memory} bytes")
print(f"Time usage:   {seconds:.2f} sec")
probs

Tree method
Memory usage: 392896 bytes
Time usage:   0.02 sec


{1: 0.3074925382704332, 0: 0.0006492506210854508, 2: 0.6918582111084819}

## Markov process

Якщо визначати всі можливі стани - то їх буде нескінченна кількість.

Кожного циклу (4 подачі) всі стани повторюються.\
Всього маємо 6 нефінальних станів та 2 фінальних (термінальних) стани.

In [20]:
score_to_pos = {(0, 0): 0,
                (0, 1): 1,
                (1, 0): 2,
                (1, 1): 3,
                (1, 2): 4,
                (2, 1): 5,
                (2, 0): 6,
                (0, 2): 7}

pos_to_score = dict(zip(score_to_pos.values(), score_to_pos.keys()))

def get_score(score):
    if sum(score)>=4:
        return (score[0]-min(score), score[1]-min(score))
    else:
        return score

def end_of_game(score):
    return abs(score[0]-score[1])>=2

def get_transition_matrix(p):
    transition_matrix = np.zeros((8, 8))
    for i in range(6):
        score = pos_to_score[i]

        new_score = (score[0]+1, score[1])
        j = score_to_pos[get_score(new_score)]
        transition_matrix[i, j] = p

        new_score = (score[0], score[1]+1)
        j = score_to_pos[get_score(new_score)]
        transition_matrix[i, j] = 1-p
            
    transition_matrix[6, 6] = 1
    transition_matrix[7, 7] = 1
    return transition_matrix

def markov_process(p):
    start = time.time()
    transition_matrix = get_transition_matrix(p)
    init_states = np.zeros((1, 8))
    init_states[0, 0] = 1
    prob_matrix = np.dot(init_states, np.linalg.matrix_power(transition_matrix, 20))
    return (sys.getsizeof(transition_matrix)+sys.getsizeof(init_states)+sys.getsizeof(prob_matrix),
            time.time()-start,
            dict(zip([0, 1, 2], [1-prob_matrix[0, 6]-prob_matrix[0, 7], prob_matrix[0, 6], prob_matrix[0, 7]])))

In [21]:
memory, seconds, probs = markov_process(p)
print("Tree method")
print(f"Memory usage: {memory} bytes")
print(f"Time usage:   {seconds:.2f} sec")
probs

Tree method
Memory usage: 1000 bytes
Time usage:   0.00 sec


{0: 0.000649250621085451, 1: 0.3074925382704353, 2: 0.6918582111084792}

## Comparison

In [22]:
def relative_error(true, est):
    return np.array([abs(true[key]-est[key])/true[key] for key in true.keys()]).mean()

In [23]:
print(f"Tree vs. Markov process: {relative_error(tree(p)[2], markov_process(p)[2])}")

Tree vs. Markov process: 3.6261158471688845e-15


In [24]:
for n in [100000, 1000000, 10000000]:
    print(f"Tree vs. Monte-Carlo(n_simulations={n}): {relative_error(tree(p)[2], monte_carlo(p, n)[2]):.4f}")

Tree vs. Monte-Carlo(n_simulations=100000): 0.1103
Tree vs. Monte-Carlo(n_simulations=1000000): 0.0203
Tree vs. Monte-Carlo(n_simulations=10000000): 0.0021


In [25]:
mc_results = []
tr_results = []
mp_results = []
for p in np.arange(0.1, 1, 0.1):
    mc = monte_carlo(p)
    tr = tree(p)
    mp = markov_process(p)
    mc_results.append((relative_error(tr[2], mc[2]), mc[0], mc[1]))
    tr_results.append((relative_error(tr[2], mp[2]), tr[0], tr[1]))
    mp_results.append((relative_error(tr[2], mp[2]), mp[0], mp[1]))

In [26]:
pd.options.display.float_format = '{:.3f}'.format
comparison_table = pd.DataFrame(columns=['Relative error', 'Memory (bytes)', 'Time (sec)'])
comparison_table.loc['Monte-Carlo'] = np.array([np.mean(np.array(i)) for i in zip(mc_results[0])])
comparison_table.loc['Tree'] = np.array([np.mean(np.array(i)) for i in zip(tr_results[0])])
comparison_table.loc['Markov process'] = np.array([np.mean(np.array(i)) for i in zip(mp_results[0])])
comparison_table

,Relative error,Memory (bytes),Time (sec)
Monte-Carlo,0.336,248.000,2.594
Tree,0.000,392896.000,0.017
Markov process,0.000,1000.000,0.000


# Task 3 (порівняння методів)

Метод Монте-Карло:
- плюси: легкий в реалізації, потребує мало пам'яті (тільки для збереження результатів)
- мінуси: невисока точність (а щоб підвищити точність, треба збільшувати кількість симуляцій -> різке збільшення часу роботи алгоритму)

Метод побудови дерева:
- плюси: точні результати, значно швидший порівняно з методом Монте-Карло, збереження порядку виграшів подач
- мінуси: потрібно багато пам'яті (вузол для кожного проміжного результату)

Метод марківського процесу:
- плюси: точні результати, висока швидкість (тільки перемноження матриць)
- мінуси: треба спочатку добре продумати множину станів та побудову матриці переходів

Для обох завдань найкращим є метод з використанням марківського процесу.

# Task 4

Головною особливістю всього тестового завдання є те, що ми знаємо ймовірності виграшу для кожного гравця.\
У реальному житті пошук цих ймовірностей теж може бути окремою складною задачею.

Перевага в економіці в кожному раунді певної команди безпосередньо може впливати на ймовірність виграшу - тобто ймовірності змінюватимуться кожного раунду.

Всі 3 методи можна застосувати в постановці задачі.
- Для методу побудови дерева - нічого складного:
на кожному раунді вираховується нова ймовірність виграшу кожної команди.
- Для марківського процесу - є особливості. Порівняно з деревом, ми не зберігаємо порядок виграшу раундів, а тільки поточний результат. В цій же грі знати послідовність може бути інформативним. Задати через стани історію попередніх раундів - виходить надбагато станів (у дереві для першої задачі їх було понад мільйон - 1 041 351 вузлів).
- Для методу Монте-Карло - та ж особливість. Не зберігається історія попередніх раундів. Але це легко реалізувати - зберігати хоча б послідовність взяття раундів для поточної карти.